In [2]:
import numpy as np
import pandas as pd
import lightkurve as lk 
import matplotlib.pyplot as plt
import astropy.table as tb
from astropy.io import ascii
import re

In [3]:
pd.options.display.max_columns = None

In [4]:
tics=ascii.read('ticresults.dat')
epics=ascii.read('epicresults.dat')

In [5]:
dtypes={'pop_id':'string','TIC':'string','gaiadr3_source_id':'string','epic_id':'string'}
id_table=pd.read_csv('idswithdiscopaper.csv',dtype=dtypes)

In [6]:
id_table=id_table[['pop_id','TIC','gaiadr3_source_id','per','epic_id','disco_paper','author','year']]

In [7]:
# id_table['tess_sectors']=""
# id_table['K2_campaign']=""

In [8]:
# id_table['TIC']=id_table['TIC'].astype('Int64').astype('string')
# id_table['epic_id']=id_table['epic_id'].astype('Int64').astype('string')

In [9]:
id_table

,pop_id,TIC,gaiadr3_source_id,per,epic_id,disco_paper,author,year
0,0,59129133,3392549449695395968,0.6253,246676629,Stauffer_2018,Stauffer,2018
1,1,59091144,3392553852037339776,3.6324,246682490,Stauffer_2018,Stauffer,2018
2,2,118769116,3411342134934571520,0.3568,247343526,Stauffer_2018,Stauffer,2018
3,3,175594111,6045818685674325760,0.5993,203354381,Stauffer_2018,Stauffer,2018
4,4,322928171,6048612514663537280,0.7794,203636498,Stauffer_2018,Stauffer,2018
...,...,...,...,...,...,...,...,...
192,192,245792696,3312903580454386048,11.4675,210554781,Rebull_2020,Rebull,2020
193,193,268510346,147940663906992512,1.4853,247888948,Rebull_2020,Rebull,2020
194,194,245869001,3313384651152556416,2.6815,246932950,Rebull_2020,Rebull,2020
195,195,60005364,147182172683187712,7.4762,247857331,Rebull_2020,Rebull,2020


In [10]:
ticdf=tics.to_pandas()

In [11]:
# for index, row in id_table.iterrows():
#     curtic=row['TIC']
#     if pd.isna(curtic):
#         continue
#     sectorlist=""
#     for index2, row2 in ticdf.iterrows():
#         if row2['target_name']==curtic:
#             sectorlist=sectorlist+row2['mission']+" "+row2['project_products']+" "+str(row2['exptime'])+"s "
#     id_table.at[index,'tess_sectors']=sectorlist

In [12]:
# id_table

In [13]:
epicdf=epics.to_pandas()

In [14]:
# for index, row in id_table.iterrows():
#     curepic=row['epic_id']
#     if pd.isna(curepic):
#         continue
#     camplist=""
#     for index2, row2 in epicdf.iterrows():
#         if re.sub(r'^\D+', '', row2['target_name'])==curepic:
#             camplist=camplist+row2['mission']+" "+row2['project_products']+" "+str(row2['exptime'])+"s "
#     id_table.at[index,'K2_campaign']=camplist

In [15]:
# id_table

In [16]:
# id_table.to_csv('availablelightcurves.csv', index=False)

In [17]:
ffis=pd.read_csv('tesstimes.csv',header=None)

In [18]:
ffis=ffis.astype(str)

In [19]:
np.unique(ffis.values.ravel(),return_counts=True)

(array(['0', '0.0', '1425.6', '158.4', '475.2'], dtype=object),
 array([ 5910, 13004,   204,   311,   271]))

In [20]:
#replace exposure times with cadence type
for row in range(ffis.shape[0]):
    for col in range(ffis.shape[1]):
        value=ffis.iat[row,col]
        if value=='158.4':
            ffis.iat[row,col]='200s'
        elif value=='475.2':
            ffis.iat[row,col]='10min'
        elif value=='1425.6':
            ffis.iat[row,col]='30min'
        else:
            ffis.iat[row,col]='none'

In [21]:
#ffis

In [22]:
np.unique(ffis.values.ravel(), return_counts=True) #counts of each match above so that's good

(array(['10min', '200s', '30min', 'none'], dtype=object),
 array([  271,   311,   204, 18914]))

In [23]:
ticdf=ticdf[ticdf['project']=='TESS'] #drop kepler observations

In [24]:
ticdf=ticdf[ticdf['exptime']==120] #only the 2min postage stamp cadences

In [25]:
ticdf['provenance_name'].unique()

array(['SPOC', 'TASOC'], dtype=object)

In [26]:
ticdf=ticdf[ticdf['provenance_name'].str.contains('SPOC')] #drop lightcurve types we don't like

In [27]:
ticdf['sector']=ticdf['mission'].str.extract(r'(\d+)').astype(int)

In [28]:
#ticdf

In [29]:
bestTESSlc=pd.DataFrame([[''] * ffis.shape[1] for _ in range(ffis.shape[0])])
bestTESSlc.columns = [f'Sector {i}' for i in range(ffis.shape[1])]

In [30]:
#needs to be in format id_type-starid-secorcamp-author-exptime so get_lc fn can work on it
# for curTIC, row in zip(id_table['TIC'],range(bestTESSlc.shape[0])):
#     if pd.isna(curTIC):
#         continue
#     for col in range(bestTESSlc.shape[1]): #remember that col is the sector number we're considering here
#         bestcurve=''
#         #first check if there's a SPOC curve for it
#         for index2, row2 in ticdf.iterrows():
#             if row2['target_name']==curTIC and row2['sector']==col:
#                 bestcurve='TIC'+'-'+curTIC+'-'+str(col)+'-'+row2['provenance_name']+'-'+str(row2['exptime'])
#         #if it didn't find a postage stamp lightcurve for this sector get the FFI info
#         if bestcurve=='': 
#             exptime=ffis.iat[row,col]
#             if exptime!='none':
#                 bestcurve='TIC'+'-'+curTIC+'-'+str(col)+'-FFI-'+exptime
#         bestTESSlc.iat[row,col]=bestcurve

In [31]:
#bestTESSlc

In [32]:
#bestTESSlc.to_csv('bestTESSlc.csv')

In [33]:
epicdf=epicdf[epicdf['project']=='K2']
epicdf=epicdf[epicdf['provenance_name']=='EVEREST']
epicdf['campaign']=epicdf['mission'].str.extract(r'(\d+)$').astype(int)

In [81]:
maxcampaign=epicdf['campaign'].max()+1

In [83]:
bestK2lc=pd.DataFrame([[''] * maxcampaign for _ in range(ffis.shape[0])])
bestK2lc.columns = [f'Campaign {i}' for i in range(maxcampaign)]

In [89]:
bestK2lc

,Campaign 0,Campaign 1,Campaign 2,Campaign 3,Campaign 4,Campaign 5,Campaign 6,Campaign 7,Campaign 8,Campaign 9,Campaign 10,Campaign 11,Campaign 12,Campaign 13,Campaign 14,Campaign 15
0,,,,,,,,,,,,,,,,
1,,,,,,,,,,,,,,,,
2,,,,,,,,,,,,,,,,
3,,,,,,,,,,,,,,,,
4,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,,,,,,,,,,,,,,,,
193,,,,,,,,,,,,,,,,
194,,,,,,,,,,,,,,,,
195,,,,,,,,,,,,,,,,


In [92]:
for curepic, row in zip(id_table['epic_id'],range(bestK2lc.shape[0])):
    if pd.isna(curepic):
        continue
    for col in range(bestK2lc.shape[1]): #remember that col is the campaign number we're considering here
        bestcurve=''
        #find EVEREST LC data
        for index2, row2 in epicdf.iterrows():
            if re.sub(r'^\D+', '', row2['target_name'])==curepic and row2['campaign']==col:
                bestcurve='EPIC'+'-'+curepic+'-'+str(col)+'-'+row2['provenance_name']+'-'+str(row2['exptime'])
        bestK2lc.iat[row,col]=bestcurve

In [98]:
bestK2lc.to_csv('bestK2lc.csv')